In [2]:
from pyspark.sql.functions import lit
df_delta= spark.sql("SELECT * FROM Lakehouse_Projet.`nyc_park_crime_delta`")

StatementMeta(, ab71e7da-8774-48fa-85ed-463b1629f2ad, 4, Finished, Available, Finished)

In [3]:
display(df_delta.limit(10))

StatementMeta(, ab71e7da-8774-48fa-85ed-463b1629f2ad, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a4adeeaf-9cbe-412a-81ce-2f699c5d5035)

In [4]:
# We don't need Total line and column as it could be calculated later
df_delta = df_delta.drop("TOTAL")
df_delta = df_delta[df_delta["PARK"] != "TOTAL"]

StatementMeta(, ab71e7da-8774-48fa-85ed-463b1629f2ad, 6, Finished, Available, Finished)

In [5]:
display(df_delta.limit(10))

StatementMeta(, ab71e7da-8774-48fa-85ed-463b1629f2ad, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f0802b93-3b2e-4ef9-ade2-f75a3432ab26)

In [6]:
df_delta.dtypes

StatementMeta(, ab71e7da-8774-48fa-85ed-463b1629f2ad, 8, Finished, Available, Finished)

[('PARK', 'string'),
 ('BOROUGH', 'string'),
 ('SIZE_ACRES', 'double'),
 ('CATEGORY', 'string'),
 ('MURDER', 'bigint'),
 ('RAPE', 'bigint'),
 ('ROBBERY', 'bigint'),
 ('FELONY_ASSAULT', 'bigint'),
 ('BURGLARY', 'bigint'),
 ('GRAND_LARCENY', 'bigint'),
 ('GRAND_LARCENY_OF_MOTOR_VEHICLE', 'bigint'),
 ('YEAR', 'int'),
 ('QUARTER', 'string')]

In [10]:
# We need to melt the dataframe to have crime cateogries in lines instead of columns as it is initially 

ids = ["PARK",
        "BOROUGH",
        "SIZE_ACRES",
        "CATEGORY",
        "YEAR",
        "QUARTER"]


df_melted = df_delta.melt(
        ids=ids, values=[col for col in df_delta.columns if col not in ids], 
        variableColumnName="CRIME_CATEGORY", 
        valueColumnName="COUNT")

StatementMeta(, ab71e7da-8774-48fa-85ed-463b1629f2ad, 12, Finished, Available, Finished)

In [11]:
df_melted.head()

StatementMeta(, ab71e7da-8774-48fa-85ed-463b1629f2ad, 13, Finished, Available, Finished)

Row(PARK='WAKEFIELD PLAYGROUND', BOROUGH='BRONX        ', SIZE_ACRES=0.877, CATEGORY='BASKETBALL & PLAYGROUND LESS THAN ONE ACRE', YEAR=2024, QUARTER='Q3', CRIME_CATEGORY='MURDER', COUNT=0)

In [8]:
from pyspark.sql.types import *
from delta.tables import *


# Create crime dimension delta table
DeltaTable.createIfNotExists(spark) \
    .tableName("dimcrimecategory") \
    .addColumn("crime_category", StringType()) \
    .addColumn("CrimeID", LongType()) \
    .execute()

StatementMeta(, ab71e7da-8774-48fa-85ed-463b1629f2ad, 10, Finished, Available, Finished)

In [13]:
from pyspark.sql.functions import *
    
# Create criùme dataframe
    
dfdimCrime = df_melted.dropDuplicates(["CRIME_CATEGORY"]).select(col("CRIME_CATEGORY")) \

# We treat it as Type 0 SCD 
# Create a surrogate key CrimeID

dfdimCrime_temp = spark.read.table("dimcrimecategory")
MAXCrimeID = dfdimCrime_temp.select(coalesce(max(col("CrimeID")),lit(0)).alias("MAXCrimeID")).first()[0]
dfdimCrime = dfdimCrime.join(dfdimCrime_temp,(dfdimCrime.CRIME_CATEGORY == dfdimCrime_temp.crime_category), "left_anti")
dfdimCrime = dfdimCrime.withColumn("CrimeID",monotonically_increasing_id() + MAXCrimeID + 1)

StatementMeta(, ab71e7da-8774-48fa-85ed-463b1629f2ad, 15, Finished, Available, Finished)

In [1]:
display(dfdimCrime.limit(10))

StatementMeta(, ab71e7da-8774-48fa-85ed-463b1629f2ad, 3, Finished, Available, Finished)

NameError: name 'dfdimCrime' is not defined

In [9]:
# Merge command is used for UPSERT operation: update records that changed and insert new values

from delta.tables import *

deltaTable = DeltaTable.forPath(spark, 'Tables/dimcrimecategory')
    
dfUpdates = dfdimCrime
    
deltaTable.alias('silver') \
  .merge(
    dfUpdates.alias('updates'),
    'silver.crime_category = updates.CRIME_CATEGORY '
  ) \
   .whenMatchedUpdate(set =
    {
          
    }
  ) \
 .whenNotMatchedInsert(values =
    {
      "crime_category": "updates.CRIME_CATEGORY",
      "CrimeID": "updates.CrimeID"
    }
  ) \
  .execute()

StatementMeta(, 3ab38269-4e18-437a-b7f5-9a5e4c2addfe, 11, Finished, Available, Finished)

In [10]:
%%sql
SELECT *
FROM dimcrimecategory
ORDER BY CrimeID
LIMIT 10;

StatementMeta(, 3ab38269-4e18-437a-b7f5-9a5e4c2addfe, 12, Finished, Available, Finished)

<Spark SQL result set with 7 rows and 2 fields>

In [14]:
# Now let's do the dimension table for parks 
# We treat this dimension as a Type 2 SCD to keep historical records 
# For example if the size of a park changes, we keep both records with same id
# Create parks dimension delta table
DeltaTable.createIfNotExists(spark) \
    .tableName("dimpark") \
    .addColumn("ParkName", StringType()) \
    .addColumn("Borough", StringType()) \
    .addColumn("Size", DecimalType(10,3)) \
    .addColumn("Category", StringType()) \
    .addColumn("ParkID", LongType()) \
    .addColumn("StartDate", TimestampType()) \
    .addColumn("EndDate", TimestampType()) \
    .addColumn("IsCurrent", BooleanType()) \
    .execute()

    
# Create parks dataframe
    
dfdimPark = df_melted.dropDuplicates(["PARK", "BOROUGH", "SIZE_ACRES", "CATEGORY"]).select(col("PARK").alias("ParkName"), col("BOROUGH").alias("current_borough"), col("SIZE_ACRES").alias("current_size"), col("CATEGORY").alias("current_category")) \

# Load existing table
dfdimPark_temp = spark.read.table("dimpark").filter("IsCurrent = true")

# Join by ParkName to detect changes
df_joined = dfdimPark.join(dfdimPark_temp, "ParkName", "left")



StatementMeta(, ab71e7da-8774-48fa-85ed-463b1629f2ad, 16, Finished, Available, Finished)

In [21]:
display(df_joined.limit(10))

StatementMeta(, 3ab38269-4e18-437a-b7f5-9a5e4c2addfe, 23, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 4257a72f-4f04-49d9-bba0-d956c91e93db)

In [15]:
# Identify changes
df_changed = df_joined.filter(
    (df_joined["Size"] != df_joined["current_size"]) | 
    (df_joined["Category"] != df_joined["current_category"]) |
    (df_joined["Borough"] != df_joined["current_borough"]) |
    df_joined["ParkID"].isNull()
)

# End-date the old record if exists
df_to_expire = df_changed.filter(df_changed["ParkID"].isNotNull()) \
    .withColumn("EndDate", current_timestamp()) \
    .withColumn("IsCurrent", lit(False))

# New version of the park
from pyspark.sql.functions import monotonically_increasing_id

max_id = dfdimPark_temp.agg({"ParkID": "max"}).collect()[0][0] or 0
df_new_versions = df_changed.select(col("ParkName"), col("current_borough").alias("Borough"), col("current_size").alias("Size"), col("current_category").alias("Category")) \
    .withColumn("ParkID", lit(max_id) + monotonically_increasing_id() + 1) \
    .withColumn("StartDate", current_timestamp()) \
    .withColumn("EndDate", lit(None).cast("timestamp")) \
    .withColumn("IsCurrent", lit(True))



StatementMeta(, ab71e7da-8774-48fa-85ed-463b1629f2ad, 17, Finished, Available, Finished)

In [16]:
display(df_new_versions.limit(10))

StatementMeta(, ab71e7da-8774-48fa-85ed-463b1629f2ad, 18, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 97284647-6fd5-4935-8670-2b8a3d62ce47)

In [17]:
# Union all changes
final_updates = df_to_expire.select(dfdimPark_temp.columns).unionByName(df_new_versions.select(dfdimPark_temp.columns))

# Upsert using DeltaTable
from delta.tables import DeltaTable
deltaTable = DeltaTable.forName(spark, "dimpark")
deltaTable.alias("t").merge(
    final_updates.alias("s"),
    "t.ParkName = s.ParkName AND t.StartDate = s.StartDate"
).whenMatchedUpdateAll() \
 .whenNotMatchedInsertAll() \
 .execute()

StatementMeta(, ab71e7da-8774-48fa-85ed-463b1629f2ad, 19, Finished, Available, Finished)

In [18]:
%%sql
SELECT *
FROM dimpark
ORDER BY ParkID
LIMIT 10;

StatementMeta(, ab71e7da-8774-48fa-85ed-463b1629f2ad, 20, Finished, Available, Finished)

<Spark SQL result set with 10 rows and 8 fields>

In [14]:
# Create date dimension delta table
DeltaTable.createIfNotExists(spark) \
    .tableName("dimdate") \
    .addColumn("Year", LongType()) \
    .addColumn("Quarter", StringType()) \
    .addColumn("DateID", LongType()) \
    .execute()

from pyspark.sql.functions import *
    
# Create criùme dataframe
    
dfdimDate = df_melted.dropDuplicates(["QUARTER", "YEAR"]).select("QUARTER", "YEAR") \

# We treat it as Type 0 SCD 
# Create a surrogate key DateID

dfdimDate_temp = spark.read.table("dimdate")
MAXDateID = dfdimDate_temp.select(coalesce(max(col("DateID")),lit(0)).alias("MAXDateID")).first()[0]
dfdimDate = dfdimDate.join(dfdimDate_temp,(dfdimDate.QUARTER == dfdimDate_temp.Quarter) & (dfdimDate.YEAR == dfdimDate_temp.Year), "left_anti")
dfdimDate = dfdimDate.withColumn("DateID",monotonically_increasing_id() + MAXDateID + 1)

StatementMeta(, d0c1b5c9-fbdb-4a7f-93e1-00312ebd238f, 16, Finished, Available, Finished)

In [17]:
deltaTable = DeltaTable.forPath(spark, 'Tables/dimdate')
    
dfUpdates = dfdimDate
    
deltaTable.alias('silver') \
  .merge(
    dfUpdates.alias('updates'),
    'silver.Year = updates.YEAR  AND silver.Quarter = updates.QUARTER'
  ) \
   .whenMatchedUpdate(set =
    {
          
    }
  ) \
 .whenNotMatchedInsert(values =
    {
      "Year": "updates.Year",
      "DateID": "updates.DateID",
      "Quarter": "updates.QUARTER"
    }
  ) \
  .execute()

StatementMeta(, d0c1b5c9-fbdb-4a7f-93e1-00312ebd238f, 19, Finished, Available, Finished)

In [18]:
%%sql
SELECT *
FROM dimdate
ORDER BY DateID
LIMIT 10;

StatementMeta(, d0c1b5c9-fbdb-4a7f-93e1-00312ebd238f, 20, Finished, Available, Finished)

<Spark SQL result set with 2 rows and 3 fields>